In [1]:
import dadi
import ipywidgets as widgets
from ipywidgets import FloatSlider, interact_manual, interact, HBox, Tab, Button, ButtonStyle, Layout

from IPython.display import display
from Models import get_dadi_model_func, get_dadi_model_params

In [2]:
# Upload data
upload = widgets.FileUpload(
               accept='',  # Accepted file extension e.g. '.fs'
               multiple=False  # True to accept multiple files upload else False
           )

In [3]:
# Select grid size
a = widgets.BoundedIntText(
    value=40,
    min=1,
    max=10000,
    step=1,
    description='Grid-x:',
    disabled=False
)
b = widgets.BoundedIntText(
    value=50,
    min=1,
    max=10000,
    step=1,
    description='Grid-y:',
    disabled=False
)
c = widgets.BoundedIntText(
    value=60,
    min=1,
    max=10000,
    step=1,
    description='Grid-z:',
    disabled=False
)

grid_ui = widgets.VBox([a, b, c])

In [4]:
# Select demographic models
model = widgets.Dropdown(
    options=['bottlegrowth_1d', 'growth_1d', 'snm_1d', 'three_epoch_1d', 'two_epoch_1d',
             'bottlegrowth_2d', 'bottlegrowth_split', 'bottlegrowth_split_mig', 'IM', 'IM_pre', 'snm_2d', 'split_mig', 'split_asym_mig'],
    description='Model:',
)

In [10]:
params = get_dadi_model_params(model.value)

# upper bounds sliders
upper_sliders = []
for p in params:
    if p == 's':
        s = widgets.BoundedFloatText(
            value=0.5,
            min=0,
            max=1,
            step=0.1,
            description=p+':',
        )
    else:
        s = widgets.BoundedFloatText(
                value=10,
                min=0,
                max=10000,
                step=0.1,
                description=p+':',
        )
    upper_sliders.append(s)
    
upper_ui = widgets.VBox(upper_sliders)

# lower bounds sliders
lower_sliders = []
for p in params:
    if p == 's':
        s = widgets.BoundedFloatText(
            value=0.5,
            min=0,
            max=1,
            step=0.1,
            description=p+':',
        )
    else:
        s = widgets.BoundedFloatText(
                value=0,
                min=0,
                max=10000,
                step=0.1,
                description=p+':',
        )
    lower_sliders.append(s)
    
lower_ui = widgets.VBox(lower_sliders)

# layout upper and lower bouunds
header  = Button(description='Lower bounds',
                 layout=Layout(width='auto', grid_area='header'),
                 style=ButtonStyle(button_color='lightblue'))
lower_ui = widgets.VBox([header, lower_ui])

header  = Button(description='Upper bounds',
                 layout=Layout(width='auto', grid_area='header'),
                 style=ButtonStyle(button_color='lightblue'))
upper_ui = widgets.VBox([header, upper_ui])

Upload frequency spectrum data, select grid size, select demographic model

In [88]:
grid1 = widgets.GridspecLayout(1, 2)

# fill it in with widgets
grid1[0:1, 0] = upload
grid1[0, 1] = grid_ui

# set the widget properties
grid1[0:1, 0].layout.height = 'auto'
grid1[0, 1].layout.height = 'auto'
grid1[0, 1].layout.width = '800px'
grid1

GridspecLayout(children=(FileUpload(value={'YRI_CEU.fs': {'metadata': {'name': 'YRI_CEU.fs', 'type': '', 'size…

In [79]:
display(model)

Dropdown(description='Model:', layout=Layout(grid_area='widget003', height='auto', width='800px'), options=('b…

Select upper and lower bounds

In [71]:
grid2 = widgets.GridspecLayout(1, 2)

# fill it in with widgets
grid2[0, 0] = upper_ui
grid2[0, 1] = lower_ui

# set the widget properties
grid2[0, 0].layout.height = 'auto'
grid2[0, 1].layout.height = 'auto'
grid2[0, 0].layout.width = 'auto'
grid2[0, 1].layout.width = 'auto'
grid2

GridspecLayout(children=(VBox(children=(Button(description='Upper bounds', layout=Layout(grid_area='header', w…

In [82]:
# Load the data from user input
uploaded_filename = next(iter(upload.value))
content = upload.value[uploaded_filename]['content']
with open(uploaded_filename, 'wb') as f: f.write(content)
# Read data into dadi
data = dadi.Spectrum.from_file(uploaded_filename)
ns = data.sample_sizes

# These are the grid point settings will use for extrapolation.
pts_l = [a.value, b.value, c.value]

func = get_dadi_model_func(model.value)
# Make the extrapolating version of our demographic model function.
func_ex = dadi.Numerics.make_extrap_log_func(func)

In [83]:
lower_bound = []
for s in lower_sliders:
    lower_bound.append(s.value)

upper_bound = []
for s in upper_sliders:
    upper_bound.append(s.value)


Interactive plots to choose initial p0

In [84]:
def plot_func_2d(**sliders_dict):    
    p0 = [x for x in sliders_dict.values()]
    print('Here are your model ('+model.value+') parameters:')
    print(p0)
    test_model = func_ex(p0, ns, pts_l)
    dadi.Plotting.plot_2d_comp_multinom(test_model, data, vmin=1, resid_range=3, show=True)

def plot_func_1d(**sliders_dict):
    p0 = [x for x in sliders_dict.values()]
    print('Here are your model ('+model.value+') parameters:')
    print(p0)
    test_model = func_ex(p0, ns, pts_l)
    dadi.Plotting.plot_1d_comp_multinom(test_model, data, vmin=1, resid_range=3, show=True)

    
sliders_dict = {}
for p, l, u in zip(params, lower_bound, upper_bound):
    sliders_dict[p+':'] = FloatSlider(
        value=0.5,
        min=l,
        max=u,
        step=0.1,
    )    

In [ ]:
if '1d' not in model.value:
    interact_manual(plot_func_2d, **sliders_dict)
else:
    interact_manual(plot_func_1d, **sliders_dict)

In [87]:
grid3 = widgets.GridspecLayout(1, 2)

# fill it in with widgets
grid3[0, 0] = interact_manual(plot_func_2d, **sliders_dict)
grid3[0, 1] = interact_manual(plot_func_1d, **sliders_dict)

# set the widget properties


interactive(children=(FloatSlider(value=0.5, description='nuB:', max=10.0), FloatSlider(value=0.5, description…

AttributeError: 'function' object has no attribute 'layout'